# Robustly Optimized BERT Pretraining Approach (RoBERTa) Notebook

```md
@authors: Grupo 03
```

In [41]:
# Notebook Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import tensorflow as tf
import shutil

from tensorflow import keras
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from enum import Enum

In [42]:
# Model run enum
class ModelRunMode(Enum):
    """
    Enumeration of Model Run Mode.
    """
    TRAIN           = "train"           # Train Mode
    CLASSIFY        = "classify"        # Classify Mode

In [43]:
# Model run mode
# Options: 
#   ModelRunMode.TRAIN.value            (Train the model)
#   ModelRunMode.CLASSIFY.value         (Classify data)
mode = ModelRunMode.CLASSIFY.value
# Prefix for saving the model files
model_prefix = "llm_roberta_model"
file_path = "llm_roberta_model_weights"
separator_char = "\t"

In [44]:
# Parameters cell
if mode == ModelRunMode.TRAIN.value:
    # TRAIN mode: Set parameters for training
    input_csv = "../tarefa_1/test_input_dataset/merged_inputs.csv"                                  # CSV file with training inputs (ID, Text)
    output_csv = "../tarefa_1/test_output_dataset/merged_outputs.csv"                               # CSV file with training outputs (ID, Label)
    test_size = 0.3                                                                                 # Proportion of the dataset to use as test data
    random_state=42                                                                                 # Seed for reproducible dataset splitting
elif mode == ModelRunMode.CLASSIFY.value:
    # CLASSIFY mode: Set parameters for classification
    input_csv = "../tarefa_3/classify_input_datasets/submission3_inputs.csv"                        # CSV file with texts for prediction (ID, Text)
    output_csv = "../tarefa_3/classify_output_datasets/submission3_outputs_llm_roberta_model.csv"   # CSV file to store prediction result
else:
    print("The selected option is not valid. Options: \"train\" or \"classify\"!")
    SystemExit()

In [45]:
# Method to load and merge two datasets by ID column
def merge_data_by_id(input_csv, output_csv, sep="\t"):
    df_in = pd.read_csv(input_csv, sep=sep)
    df_out = pd.read_csv(output_csv, sep=sep)

    # Remove duplicates or NaNs if needed
    df_in.dropna(subset=["ID", "Text"], inplace=True)
    df_out.dropna(subset=["ID", "Label"], inplace=True)
    df_in.drop_duplicates(subset=["ID"], inplace=True)
    df_out.drop_duplicates(subset=["ID"], inplace=True)

    df_merged = pd.merge(df_in, df_out, on="ID", how="inner")
    return df_merged

# Method for text cleaning
def text_cleaning(text):
        # Remove URLs
        text = re.sub(r'http[s]?://\S+', "", text)
        # Remove HTML tags
        text = re.sub(r"<[^>]*>", "", text)
        # Remove common LaTeX commands
        text = re.sub(r"\\[a-zA-Z]+(\{.*?\})?", "", text)
        # Remove email addresses
        text = re.sub(r'\S+@\S+', "", text)
        # Return cleaned text
        return text

# Method to convert labels to binary
def convert_labels_to_binary_and_text(df_merged):
    df_merged["Label"] = df_merged["Label"].str.lower().str.strip()
    y = np.where(df_merged["Label"] == "ai", 1, 0)
    texts = df_merged["Text"].tolist()
    return y, texts

# Method to plot the learning curves
def plot_learning_curves(history):
    # Loss
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title("Loss")
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Acc')
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title("Accuracy")
    plt.legend()
    plt.show()

# Method to check label distribution
def check_label_distribution(df_merged):
    label_counts = df_merged["Label"].value_counts(dropna=False)
    print("Label distribution:\n", label_counts)

# Method to print the first 5 cleaned texts
def debug_text_cleaning(df_merged):
    for i in range(min(5, len(df_merged))):
        print(df_merged["Text"].iloc[i])

In [46]:
# BERT Model training function
def train_roberta(input_csv, output_csv, model_ckpt="roberta-base", output_dir=file_path, model_prefix= model_prefix, num_train_epochs=3, batch_size=8, test_size=0.2, learning_rate=1e-3, random_state=48):
    # Load data
    df_merged = merge_data_by_id(input_csv, output_csv, sep=separator_char)

    # Check label distribution
    check_label_distribution(df_merged=df_merged)

    # Text cleaning
    df_merged["Text"] = df_merged["Text"].apply(text_cleaning)

    # Print the first 5 cleaned texts
    debug_text_cleaning(df_merged)

    # Convert label/text
    labels, texts = convert_labels_to_binary_and_text(df_merged)

    # Split entire dataset into train_val vs test
    X_train_texts, X_val_texts, y_train, y_val = train_test_split(texts, labels, test_size=test_size, random_state=random_state, stratify=labels)

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    # Tokenize without truncation/padding to compute lengths
    raw_train_encodings = tokenizer(X_train_texts, add_special_tokens=True, truncation=False, padding=False)
    token_lengths = [len(seq) for seq in raw_train_encodings["input_ids"]]

    # Calculate max_length based on the 90th percentile
    max_length = int(np.percentile(token_lengths, 90))

    # Tokenize with truncation and padding using the calculated max_length
    train_encodings = tokenizer(X_train_texts, truncation=True, padding="max_length", max_length=max_length)
    val_encodings = tokenizer(X_val_texts, truncation=True, padding="max_length", max_length=max_length)

    # Convert to tf.data.Dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), y_val))

    # Batch the datasets
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    # Load the model & set high dropout rates for regularization
    model = TFAutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)
    model.config.hidden_dropout_prob = 0.2
    model.config.attention_probs_dropout_prob = 0.2
    
    # Initially freeze base RoBERTa layers so only the classifier head trains
    model.roberta.trainable = False
    
    # Learning rate schedule & optimizer with gradient clipping
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=learning_rate,
        decay_steps=1000,
        decay_rate=0.9,
        staircase=True
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0)
    
    # Label smoothing in the loss function
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
    
    # Callbacks
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
    
    # Train
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=num_train_epochs,
        callbacks=[early_stopping]
    )

    # Evaluate on validation set
    val_loss, val_acc = model.evaluate(val_dataset)
    print(f"\nValidation Accuracy: {val_acc:.4f} | Validation Loss: {val_loss:.4f}")

    # Display model sumary
    model.summary()

    # Plot training curves
    plot_learning_curves(history)

    # If the output_dir already exists, remove it entirely
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)

    # Now recreate the folder
    os.makedirs(output_dir, exist_ok=True)

    # Construct the file paths
    model_path = os.path.join(output_dir, f"{model_prefix}_model")
    tokenizer_path = os.path.join(output_dir, f"{model_prefix}_tokenizer")
    config_path = os.path.join(output_dir, f"{model_prefix}_config.json")

    # Save the model weights
    print("Saving model to:", model_path)
    model.save_pretrained(model_path)

    # Save the model tokenizer
    print("Saving tokenizer to:", tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)

    # Save configuration
    print("Save configuration to:", config_path)
    config_data = {
        "model_ckpt": model_ckpt,
        "num_train_epochs": num_train_epochs,
        "batch_size": batch_size,
        "max_length": max_length
    }
    
    with open(config_path, "w") as f:
        json.dump(config_data, f)

    # Print end message
    print(f"RoBERTa Model, tokenizer and configuration stored under {output_dir}. Finished Training!")

In [47]:
# Classification function
def classify_roberta(input_csv, output_csv, output_dir="llm_roberta_model_weights", separator_char=separator_char):
    # Construct the file paths
    model_path = os.path.join(output_dir, f"{model_prefix}_model")
    tokenizer_path = os.path.join(output_dir, f"{model_prefix}_tokenizer")
    config_path = os.path.join(output_dir, f"{model_prefix}_config.json")

    # Load the model
    print(f"Loading model from: {output_dir}")
    model = TFAutoModelForSequenceClassification.from_pretrained(model_path)
    
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    # Loading configuration
    print("Loading configuration from:", config_path)
    with open(config_path, "r") as f:
         config_data = json.load(f)

    # Retrieve the saved configuration max_len
    max_length = config_data["max_length"]

    # Read the input CSV
    df_input = pd.read_csv(input_csv, sep=separator_char)
    if "ID" not in df_input.columns or "Text" not in df_input.columns:
        raise ValueError("Input CSV must have 'ID' and 'Text' columns for classification.")
    
    # Extract texts
    texts = df_input["Text"].astype(str).tolist()
    
    # Tokenize
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
    
    # Create tf.data.Dataset
    dataset = tf.data.Dataset.from_tensor_slices(dict(encodings))
    dataset = dataset.batch(16)
    
    # Predict probabilities (logits => sigmoid)
    outputs = model.predict(dataset)

    if isinstance(outputs, dict) and "logits" in outputs:
        logits = outputs["logits"]
    else:
        logits = outputs
    
    # Convert logits to probabilities
    probs = tf.nn.sigmoid(logits[:, 1]).numpy()
    
    # Threshold => "AI" vs "Human"
    pred_bin = (probs >= 0.5).astype(int)
    pred_labels = np.where(pred_bin == 1, "AI", "Human")
    
    # Save result
    df_out = pd.DataFrame({"ID": df_input["ID"], "Label": pred_labels})
    df_out.to_csv(output_csv, sep=separator_char, index=False)
    print(f"Predictions saved to {output_csv}")

In [48]:
if mode == ModelRunMode.TRAIN.value:
    # Check if Tensorflow is listing available GPUs (if not, continue with CPU)
    print("Tensorflow List of GPUs:", tf.config.list_physical_devices('GPU'))
    
    # Train model
    train_roberta(input_csv=input_csv, output_csv=output_csv, model_ckpt="roberta-base", output_dir=file_path, model_prefix=model_prefix, num_train_epochs=5, batch_size=16, test_size=test_size, learning_rate=5e-5, random_state=random_state)

if mode == ModelRunMode.CLASSIFY.value:
    # Classification
    classify_roberta(input_csv=input_csv, output_csv=output_csv, output_dir=file_path, separator_char=separator_char)

Loading model from: llm_roberta_model_weights


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at llm_roberta_model_weights\llm_roberta_model_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Loading configuration from: llm_roberta_model_weights\llm_roberta_model_config.json
7/7 [==============================] - 3s 103ms/step
Predictions saved to ../tarefa_3/classify_output_datasets/submission3_outputs_llm_roberta_model.csv
